# Export Best Model to ONNX Format
This notebook demonstrates how to export the trained TinyGRUVAD model to the ONNX format for deployment.

In [5]:
# Import Required Libraries
import torch
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F

# Define the TinyGRUVAD model explicitly
class TinyGRUVAD(nn.Module):
    """Light GRU-based VAD, causal, hearing-aid friendly (~3.5k params with 24 mel bands)."""
    def __init__(self, input_dim=48, hidden_dim=16, dropout=0.1):
        super().__init__()
        self.pre = nn.Conv1d(input_dim, input_dim, kernel_size=3, padding=0, groups=input_dim)
        self.norm = nn.LayerNorm(input_dim)
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, h=None):
        # x: (B,T,F)
        x = x.transpose(1,2)              # (B,F,T)
        # causal pad: pad (kernel_size-1) frames on the left only so conv doesn't see future frames
        k = self.pre.kernel_size[0] if isinstance(self.pre.kernel_size, (list, tuple)) else self.pre.kernel_size
        pad_left = k - 1
        x = F.pad(x, (pad_left, 0))       # pad on time dimension (left, right)
        x = self.pre(x).transpose(1,2)    # local causal conv
        x = self.norm(x)
        out, h = self.gru(x, h)
        out = self.drop(out)
        # return raw logits (B,T,1); use BCEWithLogitsLoss for stability
        logits = self.fc(out)
        return logits, h

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Load the Best Model
repo_root = Path.cwd().parent.parent  # Adjust path as needed
model_path = repo_root / "models" / "GRU_VAD" / "tiny_vad_best.pth"

# Initialize the model
input_dim = 48  # 24 mel bands + delta features
hidden_dim = 16
vad = TinyGRUVAD(input_dim=input_dim, hidden_dim=hidden_dim).to(device)

# Load the trained weights
vad.load_state_dict(torch.load(model_path, map_location=device))
vad.eval()

print("[INFO] Best model loaded successfully.")

[INFO] Best model loaded successfully.


In [7]:
# Define Dummy Input for ONNX Export
dummy_input = torch.randn(1, 100, input_dim).to(device)  # (Batch, Time, Features)

In [8]:
# Export the Model to ONNX Format
onnx_path = repo_root / "models" / "GRU_VAD" / "tiny_vad_best.onnx"

torch.onnx.export(
    vad, 
    dummy_input, 
    onnx_path, 
    export_params=True,  # Store the trained parameter weights inside the model file
    opset_version=11,    # ONNX version
    do_constant_folding=True,  # Optimize constant folding
    input_names=['input'],   # Input tensor name
    output_names=['output'], # Output tensor name
    dynamic_axes={
        'input': {0: 'batch_size', 1: 'sequence_length'},  # Dynamic axes for batch and sequence length
        'output': {0: 'batch_size', 1: 'sequence_length'}
    }
)

print(f"[INFO] Model exported to ONNX format at: {onnx_path}")

C:\Users\kapun_63wn2un\AppData\Local\Temp\ipykernel_32688\2729632937.py:4: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
c:\Users\kapun_63wn2un\Documents\ELEN4012 - Investigation\Repository\PROJECT-25P85\.venv\Lib\site-packages\torch\onnx\_internal\jit_utils.py:309: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\actions-runner\_work\

[INFO] Model exported to ONNX format at: c:\Users\kapun_63wn2un\Documents\ELEN4012 - Investigation\Repository\PROJECT-25P85\models\GRU_VAD\tiny_vad_best.onnx


In [9]:
# Verify the ONNX Model
import onnx

# Load the ONNX model
onnx_model = onnx.load(onnx_path)

# Check the model
onnx.checker.check_model(onnx_model)
print("[INFO] ONNX model is valid.")

[INFO] ONNX model is valid.
